# Getting data

In [1]:
import pyodbc
import pandas as pd
import os
from config.settings import DATA_DIR

# Load the Excel file
pdf = 'PO166939-204865'
data_folder = os.path.join(DATA_DIR, 'qc_data', 'qc_reports_output', pdf, 'pdf_data')
excel_file = "data.xlsx"
df = pd.read_excel(os.path.join(data_folder, excel_file), engine="openpyxl")

# post processing

In [2]:
# Convert 'customer_pallet_id' to integers, handle NaN values
df['customer_pallet_id'] = df['customer_pallet_id'].fillna(0).astype(int)

# For float columns that need precision, round them or cast to the appropriate type
df['total_defects_percentage'] = df['total_defects_percentage'].round(3)
df['estimated_yield_percentage'] = df['estimated_yield_percentage'].round(3)


df['received_date'] = pd.to_datetime(df['received_date'], errors='coerce')
df['inspection_date'] = pd.to_datetime(df['inspection_date'], errors='coerce')
df['print_date'] = pd.to_datetime(df['print_date'], errors='coerce').dt.date
df['harvest_date'] = pd.to_datetime(df['harvest_date'], errors='coerce').dt.date

In [3]:
# Establish database connection
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=RAVIKUMAR;'
    'DATABASE=DPSLocalDB;'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

# Properly format column names
columns = ', '.join([f"[{col}]" for col in df.columns])  # Handles spaces in column names
placeholders = ', '.join(['?' for _ in df.columns])  # Generates (?, ?, ?, ...)

# Define SQL INSERT statement with dynamic columns
insert_query = f"""
INSERT INTO FruitInspection ({columns}) VALUES ({placeholders})
"""

# Insert each row into the database
for _, row in df.iterrows():
    cursor.execute(insert_query, tuple(row))

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")

Data inserted successfully!
